<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import time
import torch

torch.cuda.is_available()

start = time.time()

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.tanh(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.tanh(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.tanh(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset


data_transformations = {
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),]), 

    "color_blur": transforms.Compose([  # 색깔 + 블러
     transforms.ColorJitter(brightness=(0.5, 0.9), 
                           contrast=(0.4, 0.8), 
                           saturation=(0.7, 0.9),
                           hue=(-0.2, 0.2),
                          ),
    transforms.GaussianBlur(kernel_size=(19, 19), sigma=(1.0, 2.0)),
    transforms.ToTensor()]),

    "basic": transforms.Compose([
    transforms.ToTensor()])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["crop_flip"])
train_aug_dataset2 = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["color_blur"])
                                                  
augmented_train_data = ConcatDataset((train_dataset,train_aug_dataset, train_aug_dataset2 ))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 80:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
train_start = time.time()

# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)

end = time.time()
print(f'Total: {end - start} sec')
print(f'Train: {end - train_start} sec')


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.0703125
Current benign train loss: 2.3257675170898438

Current batch: 100
Current benign train accuracy: 0.28125
Current benign train loss: 2.032287359237671

Current batch: 200
Current benign train accuracy: 0.265625
Current benign train loss: 1.9287817478179932

Current batch: 300
Current benign train accuracy: 0.34375
Current benign train loss: 1.849995732307434

Current batch: 400
Current benign train accuracy: 0.375
Current benign train loss: 1.783151626586914

Current batch: 500
Current benign train accuracy: 0.3515625
Current benign train loss: 1.7988508939743042

Current batch: 600
Current benign train accuracy: 0.4296875
Current benign train loss: 1.667525053024292

Current batch: 700
Current benign train accuracy: 0.375
Current benign train loss: 1.661342978477478

Current batch: 800
Current benign train accuracy: 0.421875
Current benign train loss: 1.56412935256958

Current batch: 900
Current benign train


Current batch: 800
Current benign train accuracy: 0.703125
Current benign train loss: 0.7738711833953857

Current batch: 900
Current benign train accuracy: 0.78125
Current benign train loss: 0.613978922367096

Current batch: 1000
Current benign train accuracy: 0.7890625
Current benign train loss: 0.743162989616394

Current batch: 1100
Current benign train accuracy: 0.8359375
Current benign train loss: 0.5492163300514221

Total benign train accuarcy: 74.20333333333333
Total benign train loss: 864.5047283470631

[ Test epoch: 5 ]

Test accuarcy: 74.39
Test average loss: 0.007413270837068558
Model Saved!

[ Train epoch: 6 ]

Current batch: 0
Current benign train accuracy: 0.7734375
Current benign train loss: 0.6275866031646729

Current batch: 100
Current benign train accuracy: 0.7578125
Current benign train loss: 0.6325846910476685

Current batch: 200
Current benign train accuracy: 0.765625
Current benign train loss: 0.6215386390686035

Current batch: 300
Current benign train accuracy: 0


Current batch: 300
Current benign train accuracy: 0.7734375
Current benign train loss: 0.5802240967750549

Current batch: 400
Current benign train accuracy: 0.7578125
Current benign train loss: 0.6676828265190125

Current batch: 500
Current benign train accuracy: 0.796875
Current benign train loss: 0.658664882183075

Current batch: 600
Current benign train accuracy: 0.8359375
Current benign train loss: 0.49102091789245605

Current batch: 700
Current benign train accuracy: 0.7890625
Current benign train loss: 0.5559762716293335

Current batch: 800
Current benign train accuracy: 0.7421875
Current benign train loss: 0.5889033079147339

Current batch: 900
Current benign train accuracy: 0.8046875
Current benign train loss: 0.4777742028236389

Current batch: 1000
Current benign train accuracy: 0.7890625
Current benign train loss: 0.6125144958496094

Current batch: 1100
Current benign train accuracy: 0.8046875
Current benign train loss: 0.4720391035079956

Total benign train accuarcy: 81.177


Current batch: 1100
Current benign train accuracy: 0.8203125
Current benign train loss: 0.464253693819046

Total benign train accuarcy: 83.62333333333333
Total benign train loss: 553.2960226237774

[ Test epoch: 16 ]

Test accuarcy: 81.2
Test average loss: 0.005557271346449852
Model Saved!

[ Train epoch: 17 ]

Current batch: 0
Current benign train accuracy: 0.84375
Current benign train loss: 0.4050429165363312

Current batch: 100
Current benign train accuracy: 0.875
Current benign train loss: 0.43915924429893494

Current batch: 200
Current benign train accuracy: 0.8671875
Current benign train loss: 0.40988507866859436

Current batch: 300
Current benign train accuracy: 0.8515625
Current benign train loss: 0.40501081943511963

Current batch: 400
Current benign train accuracy: 0.8671875
Current benign train loss: 0.4179360866546631

Current batch: 500
Current benign train accuracy: 0.8359375
Current benign train loss: 0.41337689757347107

Current batch: 600
Current benign train accuracy


Current batch: 500
Current benign train accuracy: 0.7890625
Current benign train loss: 0.545140266418457

Current batch: 600
Current benign train accuracy: 0.8671875
Current benign train loss: 0.46456724405288696

Current batch: 700
Current benign train accuracy: 0.859375
Current benign train loss: 0.437831848859787

Current batch: 800
Current benign train accuracy: 0.8671875
Current benign train loss: 0.4840949773788452

Current batch: 900
Current benign train accuracy: 0.84375
Current benign train loss: 0.39169609546661377

Current batch: 1000
Current benign train accuracy: 0.8203125
Current benign train loss: 0.4841201901435852

Current batch: 1100
Current benign train accuracy: 0.8671875
Current benign train loss: 0.35544300079345703

Total benign train accuarcy: 85.02733333333333
Total benign train loss: 506.57014621794224

[ Test epoch: 22 ]

Test accuarcy: 75.31
Test average loss: 0.008389620918035507
Model Saved!

[ Train epoch: 23 ]

Current batch: 0
Current benign train accu

...


Current batch: 500
Current benign train accuracy: 0.9765625
Current benign train loss: 0.07123264670372009

Current batch: 600
Current benign train accuracy: 0.9609375
Current benign train loss: 0.0894891545176506

Current batch: 700
Current benign train accuracy: 0.953125
Current benign train loss: 0.11937595158815384

Current batch: 800
Current benign train accuracy: 0.953125
Current benign train loss: 0.11231103539466858

Current batch: 900
Current benign train accuracy: 0.96875
Current benign train loss: 0.09661434590816498

Current batch: 1000
Current benign train accuracy: 0.9453125
Current benign train loss: 0.14177975058555603

Current batch: 1100
Current benign train accuracy: 0.9453125
Current benign train loss: 0.13233253359794617

Total benign train accuarcy: 96.534
Total benign train loss: 121.10010059922934

[ Test epoch: 72 ]

Test accuarcy: 88.71
Test average loss: 0.004047845454514027
Model Saved!

[ Train epoch: 73 ]

Current batch: 0
Current benign train accuracy: 0


Test accuarcy: 87.68
Test average loss: 0.004587034060806036
Model Saved!

[ Train epoch: 78 ]

Current batch: 0
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02430243231356144

Current batch: 100
Current benign train accuracy: 0.96875
Current benign train loss: 0.06870874762535095

Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.09720103442668915

Current batch: 300
Current benign train accuracy: 0.9765625
Current benign train loss: 0.06617716699838638

Current batch: 400
Current benign train accuracy: 0.953125
Current benign train loss: 0.12568867206573486

Current batch: 500
Current benign train accuracy: 0.9296875
Current benign train loss: 0.18368974328041077

Current batch: 600
Current benign train accuracy: 0.9765625
Current benign train loss: 0.11708170175552368

Current batch: 700
Current benign train accuracy: 0.96875
Current benign train loss: 0.07288052141666412

Current batch: 800
Current benign train accura


Current batch: 600
Current benign train accuracy: 0.9921875
Current benign train loss: 0.024481717497110367

Current batch: 700
Current benign train accuracy: 0.9765625
Current benign train loss: 0.04384305700659752

Current batch: 800
Current benign train accuracy: 0.9921875
Current benign train loss: 0.03743433579802513

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.017878307029604912

Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.016949063166975975

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.009798957966268063

Total benign train accuarcy: 98.454
Total benign train loss: 56.67428462440148

[ Test epoch: 83 ]

Test accuarcy: 90.81
Test average loss: 0.003450324732810259
Model Saved!

[ Train epoch: 84 ]

Current batch: 0
Current benign train accuracy: 0.9453125
Current benign train loss: 0.1395181119441986

Current batch: 100
Current benign train accuracy: 0.9921875
Cu


Current batch: 0
Current benign train accuracy: 0.9765625
Current benign train loss: 0.0381607748568058

Current batch: 100
Current benign train accuracy: 0.9765625
Current benign train loss: 0.05589675530791283

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.024640066549181938

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.01393261831253767

Current batch: 400
Current benign train accuracy: 0.9921875
Current benign train loss: 0.029680496081709862

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.011713128536939621

Current batch: 600
Current benign train accuracy: 0.9921875
Current benign train loss: 0.015506048686802387

Current batch: 700
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02763175778090954

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.012581986375153065

Current batch: 900
Current benign train accuracy:


Current batch: 800
Current benign train accuracy: 0.9765625
Current benign train loss: 0.053431641310453415

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.007740427274256945

Current batch: 1000
Current benign train accuracy: 0.984375
Current benign train loss: 0.04176551476120949

Current batch: 1100
Current benign train accuracy: 0.984375
Current benign train loss: 0.07086443156003952

Total benign train accuarcy: 98.856
Total benign train loss: 42.648445658851415

[ Test epoch: 94 ]

Test accuarcy: 91.23
Test average loss: 0.0034530621506273745
Model Saved!

[ Train epoch: 95 ]

Current batch: 0
Current benign train accuracy: 0.984375
Current benign train loss: 0.04119432345032692

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.05229713395237923

Current batch: 200
Current benign train accuracy: 0.9765625
Current benign train loss: 0.06492690742015839

Current batch: 300
Current benign train accuracy: 0.